In [31]:
import sys

sys.path.append("../modules")

import pushift_wrap as rapi
import os
import requests
import time
import random
import json
from datetime import datetime

data_path = os.path.join('C:/', 'data', 'reddit')

sub_end = "https://api.pushshift.io/reddit/search/submission/"
subcomment_end = "https://api.pushshift.io/reddit/submission/comment_ids/"
comment_end = "https://api.pushshift.io/reddit/search/comment/"

In [3]:
def get_submissions(ids, fields = ['author', 'created_utc', 'domain', 'full_link', 'gildings', 'id', 'is_original_content', 'is_reddit_media_domain', 
                                                                           'locked', 'media_only', 'num_comments', 'num_crossposts', 'over_18', 'permalink', 'pinned', 'post_hint', 'preview', 
                                                                           'retrieved_on', 'score', 'selftext', 'spoiler', 'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
                                                                           'subreddit_type', 'thumbnail', 'title', 'total_awards_received', 'upvote_ratio', 'url', 'url_overridden_by_dest',
                                                                           'comments']):
    
    endpoint = "https://api.pushshift.io/reddit/search/submission/"
    fields_str = ','.join(fields)
    
    request_url = f"{endpoint}?ids={ids}&fields={fields_str}"
    
    submissions = requests.get(request_url).json().get('data')
    return(submissions)

In [10]:
ids = "sdyfo5,b2yica,gv776d,jwob0l,75ar86,6lkzgn,9sf61c,fj0rl6,fj0rl6,6kt4ma,675rlh,kruyq7,i5igbn,fxyde1,glhqox,gssq9s,m2pamp,ffdf2o,krxl6t,kibtqm"
ids_list = ["sdyfo5","b2yica","gv776d","jwob0l","75ar86","6lkzgn","9sf61c","fj0rl6","fj0rl6","6kt4ma","675rlh","kruyq7","i5igbn","fxyde1","glhqox","gssq9s","m2pamp","ffdf2o","krxl6t","kibtqm"]

In [19]:
submissions = get_submissions(ids = ids)

In [20]:
missing = set(ids_list) - set([submission.get('id') for submission in submissions])

In [21]:
missing

{'fj0rl6', 'gv776d'}

In [22]:
from json import JSONDecodeError

for c, submission in enumerate(submissions, start = 1):
    progress = "|{0}| {1:.2f} %".format(("="*int(c/len(submissions) * 50)).ljust(50), c/len(submissions) * 100)
    if 'comments' in submission:
        print(progress, end = "\r")
        continue
    
    try:
        subid = submission.get('id')
        commentids = requests.get(f"{subcomment_end}{subid}").json().get('data')
    
        comments = rapi.get_comments(commentids)
        submission['comments'] = comments
    
        time.sleep(random.uniform(0.03, 0.10))
    except JSONDecodeError:
        print(progress, end = "\r")
        continue
        
    print(progress, end = "\r")

In [25]:
posts_df = pd.DataFrame.from_records(submissions)

posts_df_long = posts_df.explode('comments').reset_index(drop=True).add_prefix('post_')
posts_df_long = posts_df_long.dropna(subset = ['post_comments'])

posts_df_long = pd.merge(posts_df_long, pd.json_normalize(posts_df_long['post_comments']).add_prefix('comment_'), left_index=True, right_index=True)

posts_df_long.shape

(43806, 52)

In [29]:
outname = "reddit_polideology_top-posts_comments_long.csv"
posts_df_long.to_csv(os.path.join(data_path, outname), index = False)